In [1]:
import pickle
import numpy as np
import pandas as pd
import os
os.chdir("/usr/src/app")
# Display columns
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 500)
pd.options.display.width=None
from deepsetstats.dataset.utils import write_pickle, load_pickle

In [2]:
PATH_PLAYERS_RANK = "deepsetstats/dataset/players/parquet/players.parquet"
PATH_BIBLE_PLAYERS = "deepsetstats/dataset/players/parquet/bible_players.parquet"
PATH_FLAGS_COUNTRIES = "deepsetstats/dataset/players/flags/flags.pickle"
PATH_MAP_ID2NAME = "deepsetstats/dataset/players/pickle/map_id2name.parquet"
PATH_MAP_NAME2ID = "deepsetstats/dataset/players/parquet/map_name2id.parquet"
PATH_MAP_CNAME2ID = "deepsetstats/dataset/players/parquet/map_cname2id.parquet"
PATH_MAPID2COUNTRY = "deepsetstats/dataset/players/parquet/map_id2country.parquet"

In [3]:
d1 = load_pickle(PATH_MAP_ID2NAME)
d2 = load_pickle(PATH_MAP_NAME2ID)
d3 = load_pickle(PATH_MAP_CNAME2ID)
d4 = load_pickle(PATH_MAPID2COUNTRY)

FileNotFoundError: [Errno 2] No such file or directory: 'deepsetstats/dataset/players/parquet/map_name2id.parquet'

In [4]:
dfp = pd.read_parquet(PATH_PLAYERS_RANK, engine="pyarrow")

In [6]:
dfp[dfp["name"] == "Novak Djokovic"]

,name,year_birth,country,ranking,date_rank
1,Novak Djokovic,1987,SRB,2,2023-07-31
0,Novak Djokovic,1987,SRB,1,2021-08-09
0,Novak Djokovic,1987,SRB,1,2015-10-05
3,Novak Djokovic,1988,SRB,4,2009-05-11
678,Novak Djokovic,1987,SRB,679,2003-09-29


In [7]:
dfb = pd.read_parquet(PATH_BIBLE_PLAYERS, engine="pyarrow")

In [10]:
dfb[dfb["best_ranking"] == 1]

,country,best_ranking,name,first_name,common_name,common_name_2,player_id
0,ESP,1,Carlos Alcaraz,Carlos,Alcaraz,Alcaraz,0
1,SRB,1,Novak Djokovic,Novak,Djokovic,Djokovic,1
138,ESP,1,Rafael Nadal,Rafael,Nadal,Nadal,138
4165,ESP,1,Juan Carlos Ferrero,Juan,Carlos Ferrero,Ferrero,4165
6804,USA,1,Pete Sampras,Pete,Sampras,Sampras,6804
8384,SWE,1,Stefan Edberg,Stefan,Edberg,Edberg,8384
8386,USA,1,Ivan Lendl,Ivan,Lendl,Lendl,8386


In [80]:
#df[df["best_ranking"] < 100].head(30)

# Extract Player title development

In [81]:
from copy import copy
import string
import regex
import re
import numpy as np

In [82]:
df = dfb[dfb["best_ranking"] < 100].copy()

In [83]:
df[df["common_name_2"] == "Ferrero"]

,country,best_ranking,name,first_name,common_name,common_name_2,player_id
4165,ESP,1,Juan Carlos Ferrero,Juan,Carlos Ferrero,Ferrero,4165


In [84]:
# Set of player names
player_common_names = df["common_name"].tolist()
player_common_names_2 = df["common_name_2"].tolist()
player_full_names = df["name"].tolist()
player_ids = df["player_id"].tolist()

In [85]:
# Define the translation table
exclude_punctuation = string.punctuation
translator = str.maketrans(exclude_punctuation, ' ' * len(exclude_punctuation))

In [117]:
# Given string
#raw_string = """It's Auger Aliassime Paul Henri Mathieu plays againts Juan Monaco, 
# CCarlos Alcaraz and Raonic advance, Opelka Takes On Tiafoe; Fils surprises; Federer 
# continues; Monfils, Hurkacz Feature | Toronto 2023 Highlights Day 1"""

raw_string = "Alcaraz Battles Purcell; Djokovic Plays Fritz | Cincinnati 2023 Quarter-Final Highlights"

In [126]:
#def extract_players_title(raw_string, player_ids, player_full_names, player_common_names, player_common_names_2):
# ------------------------------------------------ #
#   Input string cleansing
# ------------------------------------------------ #
# Remove punctuation and replace with spaces
query_string = raw_string.translate(translator)

# Replace consecutive spaces with single spaces
query_string = re.sub(r'\s+', ' ', query_string)

player_ids_candidates_in_query = []
fullname_candidates_in_query = []
common_name_candidates_in_query = []

iter_string = copy(query_string)
for full_name, common_name, common_name2, player_id in zip(
    player_full_names, player_common_names, player_common_names_2, player_ids
):
    # Remove punctuation and replace with spaces
    full_name = full_name.translate(translator)
    common_name = common_name.translate(translator)
    common_name2 = common_name2.translate(translator)

    # Quick check if the surname it's there
    #par_rat1 = fuzz.partial_ratio(iter_string, common_name)
    #par_rat2 = fuzz.partial_ratio(iter_string, common_name2)
    min_len = np.min([len(common_name), len(common_name2)])
    permitted_errors = 0 if min_len < 5 else 1
    match_common_1 = regex.search(fr'(\s|^)({common_name})(\s){{e<={permitted_errors}}}', query_string)
    match_common_2 = regex.search(fr'(\s|^)({common_name2})(\s){{e<={permitted_errors}}}', query_string)

    if match_common_1 or match_common_2:
        cname = common_name if match_common_1 else common_name2
        idx_common_name_max = 0 if match_common_1 else 0

        player_ids_candidates_in_query.append(player_id)
        fullname_candidates_in_query.append(full_name)
        
        # print(f"Player id: {player_id}, Common_name: {cname}, Permitted errors: {permitted_errors}")
        common_name_candidates_in_query.append(cname)

player_ids_in_query = []
iter_player_ids_candidates_in_query = copy(player_ids_candidates_in_query)
iter_common_name_candidates_in_query = copy(common_name_candidates_in_query)

for full_name, common_name, player_id in zip(
    fullname_candidates_in_query, common_name_candidates_in_query, player_ids_candidates_in_query):
    
    # Remove punctuation and replace with spaces
    full_name = full_name.translate(translator)
    common_name = common_name.translate(translator)
    
    # Check if the full name is there
    full_name_match = regex.search(fr'({full_name}){{e<=1}}', iter_string)

    # If full name is there
    if full_name_match:
        start, end = full_name_match.span()
        iter_string = iter_string[:start] + iter_string[end:]
        # print(f"{full_name} player: full name trimmed at:", start, end)
        
        # Add candidate as the final match of player found in video title
        player_ids_in_query.append(player_id)

        # Remove candidate
        iter_player_ids_candidates_in_query.remove(player_id)
        iter_common_name_candidates_in_query.remove(common_name)
        continue

# For the rest, get the best common name match
for common_name, player_id in zip(
    iter_common_name_candidates_in_query, iter_player_ids_candidates_in_query
):
    # Remove punctuation and replace with spaces
    common_name = common_name.translate(translator)
    
    # Check that the entire common name is surrounded by spaces or commas
    cname_name_match = regex.search(fr'(^|\s)({common_name})($|\s){{e<=1}}', iter_string)

    # If full name is there
    if cname_name_match:
        start, end = cname_name_match.span()
        iter_string = iter_string[:start] + " " + iter_string[end:]
        # print(f"{common_name} player: cname name trimmed at:", start, end)
        
        # Add candidate as the final match of player found in video title
        player_ids_in_query.append(player_id)

        # Remove candidate
        player_ids_candidates_in_query.remove(player_id)

In [127]:
query_string

'Alcaraz Battles Purcell Djokovic Plays Fritz Cincinnati 2023 Quarter Final Highlights'

In [128]:
[d1[xx] for xx in iter_player_ids_candidates_in_query]

['Carlos Alcaraz', 'Novak Djokovic', 'Taylor Fritz', 'Max Purcell']

In [90]:
dfb[dfb["common_name_2"] == "Fils"]

,country,best_ranking,name,first_name,common_name,common_name_2,player_id
46,FRA,47,Arthur Fils,Arthur,Fils,Fils,46


In [98]:
[d1[xx] for xx in player_ids_candidates_in_query]

['Carlos Alcaraz',
 'Tommy Paul',
 'Arthur Fils',
 'Juan Monaco',
 'Paul-Henri Mathieu']

# Test in a function


In [109]:
from deepsetstats.dataset import utils
from importlib import reload

In [110]:
utils = reload(utils)

In [111]:
ept = utils.extract_players_title

In [114]:
%%time
# Given string
raw_string = """Alcaraz Battles Purcell; Djokovic Plays Fritz | Cincinnati 2023 Quarter-Final Highlights"""

pids = ept(
    raw_string=raw_string,
    player_ids=player_ids,
    player_full_names=player_full_names,
    player_common_names=player_common_names,
    player_common_names_2=player_common_names_2,
)

CPU times: user 148 ms, sys: 4.38 ms, total: 153 ms
Wall time: 152 ms


In [115]:
[d1[xx] for xx in pids]

['Novak Djokovic', 'Taylor Fritz', 'Max Purcell']